In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,27800
2,Huelva,Confirmados PDIA 14 días,1262
3,Huelva,Tasa PDIA 14 días,"245,92240388175458"
4,Huelva,Confirmados PDIA 7 días,582
5,Huelva,Total Confirmados,27973
6,Huelva,Curados,23387
7,Huelva,Fallecidos,360


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  27800.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7215.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1013 personas en los últimos 7 días 

Un positivo PCR cada 508 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,27800.0,582.0,1262.0,513170.0,113.412709,245.922404,217.0
Condado-Campiña,8587.0,300.0,638.0,156231.0,192.023350,408.369658,119.0
Huelva-Costa,16318.0,219.0,501.0,289548.0,75.635128,173.028306,67.0
Almonte,1415.0,107.0,246.0,24507.0,436.609948,1003.794834,50.0
Huelva (capital),7215.0,142.0,283.0,143837.0,98.722860,196.750488,35.0
Sierra de Huelva-Andévalo Central,2479.0,57.0,106.0,67391.0,84.581027,157.291033,31.0
Moguer,1046.0,46.0,89.0,21867.0,210.362647,407.005991,20.0
Rociana del Condado,489.0,32.0,69.0,7939.0,403.073435,869.127094,14.0
Bollullos Par del Condado,760.0,28.0,53.0,14387.0,194.620143,368.388128,14.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cumbres Mayores,43.0,18.0,18.0,1749.0,1029.159520,1029.159520,10.0
Almonte,1415.0,107.0,246.0,24507.0,436.609948,1003.794834,50.0
Rociana del Condado,489.0,32.0,69.0,7939.0,403.073435,869.127094,14.0
Santa Ana la Real,9.0,4.0,4.0,475.0,842.105263,842.105263,3.0
Cumbres de San Bartolomé,8.0,3.0,3.0,376.0,797.872340,797.872340,NaN
Cartaya,1234.0,38.0,126.0,20083.0,189.214759,627.396305,13.0
Galaroza,24.0,8.0,8.0,1382.0,578.871201,578.871201,6.0
Almonaster la Real,63.0,6.0,9.0,1788.0,335.570470,503.355705,1.0
Lucena del Puerto,191.0,4.0,15.0,3261.0,122.661760,459.981601,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Ayamonte,1262.0,1.0,7.0,21104.0,4.738438,33.169067,1.0,0.142857
Minas de Riotinto,107.0,1.0,7.0,3812.0,26.232949,183.630640,0.0,0.142857
Lepe,2061.0,6.0,23.0,27880.0,21.520803,82.496413,2.0,0.260870
Lucena del Puerto,191.0,4.0,15.0,3261.0,122.661760,459.981601,0.0,0.266667
Palos de la Frontera,563.0,12.0,44.0,11742.0,102.197241,374.723216,3.0,0.272727
Aracena,502.0,7.0,24.0,8255.0,84.797093,290.732889,3.0,0.291667
Beas,175.0,3.0,10.0,4341.0,69.108500,230.361668,1.0,0.300000
Cartaya,1234.0,38.0,126.0,20083.0,189.214759,627.396305,13.0,0.301587
Aljaraque,986.0,10.0,26.0,21474.0,46.567943,121.076651,5.0,0.384615
